In [ ]:
from google.colab import files

uploaded = files.upload()

Saving memo_dataset7.csv to memo_dataset7.csv


In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import kagglehub
import kagglehub
import seaborn as sns
from scipy import signal
from dateutil.parser import parse
from pandas.plotting import autocorrelation_plot
from google.colab import drive

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', None)

df = pd.read_csv('memo_dataset7.csv')

columns = df.columns.tolist()
new_order = [columns[9]] + columns[:9] + columns[9:]
df = df[new_order]
df = df.loc[:, ~df.columns.duplicated()]

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
import torch
import numpy as np
import random

seed_value = 42

torch.manual_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [ ]:
df = df[['MCP (Euro)', 'co', 'no', 'no2', 'o3', 'so2', 'pm2_5', 'pm10', 'nh3', 'MCV (MWh)', 'temp', 'dwpt', 'rhum', 'prcp', 'wspd', 'wpgt', 'pres','MCP_lag1', 'MCP_lag2', 'MCP_lag3', 'Trend', 'Seasonal', 'Residual',
 'year', 'hour', 'quarter', 'dayofyear', 'month', 'dayofweek']]
df.head()

,MCP (Euro),co,no,no2,o3,so2,pm2_5,pm10,nh3,MCV (MWh),temp,dwpt,rhum,prcp,wspd,wpgt,pres,MCP_lag1,MCP_lag2,MCP_lag3,Trend,Seasonal,Residual,year,hour,quarter,dayofyear,month,dayofweek
0,54.998618,216.96,0.12,1.30,98.71,5.90,3.83,3.97,0.01,17.6,23.2,14.7,59.0,0.0,7.2,14.8,1014.4,54.998618,56.178388,71.000894,79.468248,0.701691,0.986307,2023,12,2,152,6,3
1,54.998293,216.96,0.10,1.30,95.84,5.48,3.55,3.68,0.04,13.4,21.2,14.8,67.0,0.0,8.3,16.7,1014.2,54.998618,54.998618,56.178388,79.495157,0.699185,0.989501,2023,13,2,152,6,3
2,54.998455,213.62,0.08,1.44,94.41,5.60,3.91,4.07,0.03,14.2,21.2,14.6,66.0,0.0,9.0,16.7,1013.9,54.998293,54.998618,54.998618,79.530954,0.730721,0.946374,2023,14,2,152,6,3
3,54.998618,213.62,0.07,1.71,90.84,5.90,4.38,4.55,0.02,8.0,21.8,14.2,62.0,0.0,7.2,18.5,1013.7,54.998455,54.998293,54.998618,79.718375,0.806774,0.855149,2023,15,2,152,6,3
4,54.999431,216.96,0.08,3.30,85.12,7.33,4.88,5.13,0.27,2.7,24.2,12.2,47.0,0.0,7.2,16.7,1011.9,54.998618,54.998455,54.998293,79.938611,0.934643,0.736133,2023,16,2,152,6,3


In [ ]:
import pandas as pd
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller

In [ ]:
data = df.copy()

In [ ]:
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

In [ ]:
def train_test_split(df, test_size=0.2):
    data = df.values

    feature_scaler.fit(data[:, :0])
    target_scaler.fit(data[:, 0:])
    scaled_data = feature_scaler.transform(data[:, :0])
    scaled_target = target_scaler.transform(data[:, 0:])
    data_scaled = np.concatenate((scaled_data, scaled_target), axis=1)


    n = int(len(data_scaled) * (1 - test_size))
    return data_scaled[:n], data_scaled[n:]

In [ ]:
def xgb_prediction(train, value):
    train = np.array(train)
    X, Y = train[:, :0], train[:, 0]
    global model
    model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

    model.fit(X, Y)
    val = np.array(value).reshape(1, -1)
    prediction = model.predict(val)
    return prediction[0]

In [ ]:
def walk_forward_validation(data, percentage=0.2):
    train, test = train_test_split(data, percentage)
    predictions = []
    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_Y = test[i, :0], test[i, 0]
        pred = xgb_prediction(history, test_X)
        predictions.append(pred)
        history.append(test[i])
    Y_test = target_scaler.inverse_transform(test[:, -1:].reshape(1, -1))
    Y_pred = target_scaler.inverse_transform(np.array(predictions).reshape(1, -1))
    test_rmse = mean_squared_error(Y_test, Y_pred, squared=False)

    return test_rmse, Y_test, Y_pred

In [ ]:
test_rmse, Y_test, predictions = walk_forward_validation(data, 0.2)

ValueError: Found array with 0 feature(s) (shape=(10957, 0)) while a minimum of 1 is required by MinMaxScaler.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from xgboost import XGBRegressor
import numpy as np

data = df.copy()
feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

# def train_test_split(df, test_size=0.2):
#     data = df.values
#     features, target = data[:, 1:], data[:, 0]
#     feature_scaler.fit(features)
#     target_scaler.fit(target.reshape(-1, 1))

#     scaled_features = feature_scaler.transform(features)
#     scaled_target = target_scaler.transform(target.reshape(-1, 1))

#     data_scaled = np.hstack((scaled_target, scaled_features))
#     n = int(len(data_scaled) * (1 - test_size))
#     return data_scaled[:n], data_scaled[n:]

def train_test_split(df):
    data = df.values
    features, target = data[:, 1:], data[:, 0]

    feature_scaler.fit(features)
    target_scaler.fit(target.reshape(-1, 1))

    scaled_features = feature_scaler.transform(features)
    scaled_target = target_scaler.transform(target.reshape(-1, 1))

    data_scaled = np.hstack((scaled_target, scaled_features))

    train = data_scaled[:10000]
    test = data_scaled[10000:]

    return train, test

def xgb_prediction(train, value):
    train = np.array(train)
    X, Y = train[:, 1:], train[:, 0]
    model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

    model.fit(X, Y)
    val = np.array(value).reshape(1, -1)
    prediction = model.predict(val)
    return prediction[0]

def walk_forward_validation(data):
    train, test = train_test_split(data)
    predictions = []
    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_Y = test[i, 1:], test[i, 0]
        pred = xgb_prediction(history, test_X)
        predictions.append(pred)
        history.append(test[i])

    Y_test = target_scaler.inverse_transform(test[:, 0].reshape(-1, 1))
    Y_pred = target_scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    test_rmse = (mean_squared_error(Y_test, Y_pred))
    mae = mean_absolute_error(Y_test, Y_pred)

    return test_rmse, Y_test, Y_pred, mae

In [ ]:
test_rmse, Y_test, predictions, abs_errors = walk_forward_validation(data)

print("Test MSE:", test_rmse)
print("Absolute Errors:", abs_errors)

Test MSE: 524.7302712153027
Absolute Errors: 6.863901600086809


In [ ]:
mae = np.mean(abs_errors)

print("Test MSE:", test_rmse)
print("Mean Absolute Error (MAE):", mae)

Test MSE: 524.7302712153027
Mean Absolute Error (MAE): 6.863901600086809
